# Estudio Comparativo de Algoritmos para Resolver el Cubo de Rubik

Este notebook presenta un estudio comparativo de tres algoritmos de búsqueda para resolver el Cubo de Rubik a una profundidad menor o igual a 10 movimientos. Los algoritmos analizados son: Búsqueda de Costo Uniforme (UCS), Algoritmo A* y Algoritmo IDA*. La comparación se basa en los criterios de Memoria, Tiempo, Calidad de la Solución y Completez. Además, se incluye una sección para proponer y aplicar una modificación a alguno de los algoritmos o heurísticas.

**Algoritmos a Comparar:**
1.  UCS (Uniform Cost Search)
2.  A* (A-Star Search) con heurística basada en Pattern Databases.
3.  IDA* (Iterative Deepening A-Star) con heurística basada en Pattern Databases.

**Criterios de Evaluación:**
- Memoria consumida.
- Tiempo de ejecución.
- Calidad de la solución (longitud de la ruta encontrada).
- Completez (capacidad de encontrar una solución si existe).

**Heurística Utilizada:**
Se empleará una heurística combinada basada en dos bases de datos de patrones (Pattern Databases - PDBs): una para las esquinas y otra para los bordes (incluyendo el centro, según la implementación original).

In [1]:
# Importar las librerías necesarias
from functools import reduce
from termcolor import colored
from random import choice, seed
from itertools import product
from abc import ABC, abstractmethod
import heapq
from collections import deque
import math
import time
import sys
import matplotlib.pyplot as plt

# ----------- Definiciones Base del Cubo de Rubik -----------

# La clase abstracta EstadoProblema
class EstadoProblema(ABC):
    """
    La clase EstadoProblema es abstracta.
    Representa un estado o configuración del problema a resolver.
    """
    @abstractmethod
    def expand(self):
        """
        :return: el conjunto de estados sucesores
        """
        pass

    @abstractmethod
    def get_depth(self):
        """
        :return: la profundidad del estado
        """
        pass

    @abstractmethod
    def get_parent(self):
        """
        :return: referencia al estado predecesor o padre
        """
        pass


# Códigos de los colores (usados en la representación de bits)
W = 0; G = 1; R = 2; B = 3; C = 4; Y = 5; # Cian para Naranja

# Diccionario con los nombres para los códigos (usados por termcolor)
color_map = {
    0:"white", 1:"green", 2:"red", 3:"blue", 4:"cyan", 5:"yellow"
}

# Mapping de letras a posiciones en bits y color original del centro de la cara
# Cada posición ocupa 3 bits en un entero largo.
code = {
    'A' : (0*3,W), 'B' : (1*3,W), 'C' : (2*3,W),
    'D' : (3*3,W), 'E' : (4*3,W), 'F' : (5*3,W),
    'G' : (6*3,W), 'H' : (7*3,W), 'I' : (8*3,W),

    'J' : (9*3,G), 'K' : (10*3,G), 'L' : (11*3,G),
    'M' : (12*3,R), 'N' : (13*3,R), 'Ñ' : (14*3,R),
    'O' : (15*3,B), 'P' : (16*3,B), 'Q' : (17*3,B),
    'R' : (18*3,C), 'S' : (19*3,C), 'T' : (20*3,C),

    'U' : (21*3,G), 'V' : (22*3,G), 'W' : (23*3,G),
    'X' : (24*3,R), 'Y' : (25*3,R), 'Z' : (26*3,R),
    'a' : (27*3,B), 'b' : (28*3,B), 'c' : (29*3,B),
    'd' : (30*3,C), 'e' : (31*3,C), 'f' : (32*3,C),

    'g' : (33*3,G), 'h' : (34*3,G), 'i' : (35*3,G),
    'j' : (36*3,R), 'k' : (37*3,R), 'l' : (38*3,R),
    'm' : (39*3,B), 'n' : (40*3,B), 'ñ' : (41*3,B),
    'o' : (42*3,C), 'p' : (43*3,C), 'q' : (44*3,C),

    'r' : (45*3,Y), 's' : (46*3,Y), 't' : (47*3,Y),
    'u' : (48*3,Y), 'v' : (49*3,Y), 'w' : (50*3,Y),
    'x' : (51*3,Y), 'y' : (52*3,Y), 'z' : (53*3,Y)
}

# Acciones (Giros de 90 grados por cara). Representan permutaciones de posiciones.
# La estructura es [eje][cara_indice][mapeo_posiciones (destino, origen)]
# No, el mapeo es (origen, destino), significa que el color de 'origen' se mueve a la posición 'destino'.
# El sentido (0 o 1) determina si se usa el mapeo directo o su inverso.
actions = [
    [ # Eje 0
        [('A','g'),('B','U'),('C','J'),('Q','A'),('c','B'),
         ('ñ','C'),('z','Q'),('y','c'),('x','ñ'),('g','z'),
         ('U','y'),('J','x'),('R','T'),('S','f'),('T','q'),
         ('d','S'),('f','p'),('o','R'),('p','d'),('q','o')],
        [('G','i'),('H','W'),('I','L'),('O','G'),('a','H'),
         ('m','I'),('t','O'),('s','a'),('r','m'),('i','t'),
         ('W','s'),('L','r'),('M','j'),('N','X'),('Ñ','M'),
         ('Z','N'),('l','Ñ'),('k','Z'),('j','l'),('X','k')]
    ],
    [ # Eje 1
        [('A','q'),('D','f'),('G','T'),('M','A'),('X','D'),
         ('j','G'),('r','M'),('u','X'),('x','j'),('T','x'),
         ('f','u'),('q','r'),('J','g'),('K','U'),('L','J'),
         ('U','h'),('W','K'),('g','i'),('h','W'),('i','L')],
        [('C','o'),('F','d'),('I','R'),('Ñ','C'),('Z','F'),
         ('l','I'),('t','Ñ'),('w','Z'),('z','l'),('o','t'),
         ('d','w'),('R','z'),('O','Q'),('P','c'),('Q','ñ'),
         ('a','P'),('c','n'),('m','O'),('n','a'),('ñ','m')]
    ],
    [ # Eje 2
        [('J','R'),('K','S'),('L','T'),('M','J'),('N','K'),
         ('Ñ','L'),('O','M'),('P','N'),('Q','Ñ'),('R','O'),
         ('S','P'),('T','Q'),('G','A'),('H','D'),('I','G'),
         ('F','H'),('C','I'),('B','F'),('A','C'),('D','B')],
        [('g','o'),('h','p'),('i','q'),('j','g'),('k','h'),
         ('l','i'),('m','j'),('n','k'),('ñ','l'),('o','m'),
         ('p','n'),('q','ñ'),('r','x'),('s','u'),('t','r'),
         ('u','y'),('w','s'),('x','z'),('y','w'),('z','t')]
    ]
]

# calcula la configuración ordenada del cubo (estado objetivo) en bits
initial_conf = reduce(lambda x,y:(0,x[1]|(y[1]<<y[0])), [(0,0)]+[v for k,v in code.items()])[1]

# Espacios en blanco y caracteres para la impresión visual del cubo
BLANK = ' '*6
FILL = 9608
K = 2

In [2]:
# ----------- Clase RubikPuzzle -----------

# La clase RubikPuzzle
class RubikPuzzle(EstadoProblema):
    """
    Cubo de Rubik de 3 X 3. Implementación usando un entero largo para la configuración de bits.
    Hereda de EstadoProblema para ser compatible con los algoritmos de búsqueda.
    """
    def __init__(self, parent = None, action=None, depth=0, configuration=None):
        """
        Inicializa un estado del cubo.
        :param parent: Estado predecesor.
        :param action: Acción aplicada para llegar a este estado desde el padre.
        :param depth: Profundidad en el árbol de búsqueda.
        :param configuration: Configuración de bits opcional para inicializar directamente.
        """
        self.parent = parent
        self.depth = depth
        self.action_from_parent = action

        if configuration is not None:
            self.configuration = configuration
        elif parent is not None and action is not None:
            # Copia la configuración del padre y aplica la acción.
            # Importante: La acción muta 'self.configuration'.
            self.configuration = parent.configuration
            self.apply(action)
        else:
            # Estado inicial por defecto: cubo resuelto.
            self.configuration = initial_conf

    # Método initialize (menos usado en el flujo principal, basado en diccionario)
    def initialize(self, pattern):
         if not all(key in code for key in pattern):
              raise ValueError("Pattern contains invalid cube positions (letters)")
         conf_bits = 0
         for key, val in pattern.items():
             pos_bits = code[key][0]
             conf_bits |= (val << pos_bits)
         self.configuration = conf_bits
         return self.configuration

    # Método para obtener la representación visual de un subcubo por su letra de posición
    def cube(self, symbol):
        """
        Obtiene la representación visual (color) de un subcubo en una posición dada.
        """
        if symbol not in code:
            return "   "
        n = code[symbol][0] # Posición de inicio en bits para este subcubo
        color_mask = 7 << n # Máscara de 3 bits para el color
        color_code = (self.configuration & color_mask) >> n # Extrae el código de color

        color_name = color_map.get(color_code, "white") # Obtiene el nombre del color
        return colored(chr(FILL), color_name) * K # Retorna el caracter coloreado

    # Método para aplicar un movimiento (acción) al cubo
    def apply(self, action):
        """
        Aplica una acción (giro de cara) a la configuración actual.
        Modifica la configuración de bits del cubo.
        :param action: Tupla (eje, cara_indice, direccion), ej: (0, 0, 0).
        """
        eje, cara_indice, direccion = action
        moves = actions[eje][cara_indice]

        # Creamos un diccionario temporal para almacenar los colores ANTES de cambiarlos,
        # mapeando la posición de DESTINO (en el sentido de la acción) al color que LLEGA allí.
        colors_to_move = {}
        if direccion == 0: # Sentido 0 (ej. horario)
            for origin_symbol, dest_symbol in moves:
                 origin_pos_bits = code[origin_symbol][0]
                 color_mask = 7 << origin_pos_bits
                 color_value = (self.configuration & color_mask) >> origin_pos_bits
                 colors_to_move[dest_symbol] = color_value # El color de origin_symbol llega a dest_symbol
        else: # Sentido 1 (ej. anti-horario)
             # En el sentido inverso, el color que llega a la posición 'origin_symbol'
             # en la lista de moves (del sentido 0) viene de la posición 'dest_symbol'
             # en esa misma lista de moves.
             for origin_symbol, dest_symbol in moves:
                 origin_pos_bits_for_inverse_movement = code[dest_symbol][0] # El color a mover está en la pos destino del sentido 0
                 color_mask = 7 << origin_pos_bits_for_inverse_movement
                 color_value = (self.configuration & color_mask) >> origin_pos_bits_for_inverse_movement

                 dest_symbol_for_inverse_movement = origin_symbol # El color llega a la pos origen del sentido 0
                 colors_to_move[dest_symbol_for_inverse_movement] = color_value


        # Aplicamos los cambios basándonos en los colores que deben llegar a cada posición.
        new_configuration = self.configuration
        # Creamos una máscara de 3 bits (111 binario) para borrar los bits de color en las posiciones de destino.
        clear_mask_template = 7

        for dest_symbol, color_value in colors_to_move.items():
             dest_pos_bits = code[dest_symbol][0]

             # Construimos la máscara para limpiar solo los bits en la posición de destino
             clear_mask = clear_mask_template << dest_pos_bits
             # Aplicamos la máscara NOT para limpiar esos bits en la nueva configuración
             new_configuration &= ~clear_mask

             # Colocamos el nuevo color en la posición de destino
             new_configuration |= (color_value << dest_pos_bits)

        self.configuration = new_configuration


    # Método para obtener la representación visual 2D del cubo (para imprimir)
    def __str__(self):
        """
        Representación visual 2D del cubo en texto coloreado.
        """
        return (
            '\n' +
            BLANK + self.cube('A') + self.cube('B') + self.cube('C') + '\n' +
            BLANK + self.cube('D') + self.cube('E') + self.cube('F') + '\n' +
            BLANK + self.cube('G') + self.cube('H') + self.cube('I') + '\n' +
            self.cube('J') + self.cube('K') + self.cube('L') +
            self.cube('M') + self.cube('N') + self.cube('Ñ') +
            self.cube('O') + self.cube('P') + self.cube('Q') +
            self.cube('R') + self.cube('S') + self.cube('T') + '\n' +
            self.cube('U') + self.cube('V') + self.cube('W') +
            self.cube('X') + self.cube('Y') + self.cube('Z') +
            self.cube('a') + self.cube('b') + self.cube('c') +
            self.cube('d') + self.cube('e') + self.cube('f') + '\n' +
            self.cube('g') + self.cube('h') + self.cube('i') +
            self.cube('j') + self.cube('k') + self.cube('l') +
            self.cube('m') + self.cube('n') + self.cube('ñ') +
            self.cube('o') + self.cube('p') + self.cube('q') + '\n' +
            BLANK + self.cube('r') + self.cube('s') + self.cube('t') + '\n' +
            BLANK + self.cube('u') + self.cube('v') + self.cube('w') + '\n' +
            BLANK + self.cube('x') + self.cube('y') + self.cube('z') + '\n'
        )

    def __repr__(self):
        return self.__str__()

    # Métodos para comparación y hashing (basados en la configuración de bits)
    def __eq__(self,other):
        if not isinstance(other, self.__class__): return False
        return self.configuration==other.configuration

    def __ne__(self,other):
        return not self.__eq__(other)

    def __lt__(self,other):
        if not isinstance(other, self.__class__): return NotImplemented
        return self.depth < other.depth

    def __hash__(self):
        return hash(self.configuration)

    # Métodos relacionados con Pattern Databases
    def pattern_equals(self, pattern_mask, target_configuration=initial_conf):
        """Compara si las partes del cubo definidas por la máscara coinciden con el objetivo."""
        current_pattern_bits = self.configuration & pattern_mask
        target_pattern_bits = target_configuration & pattern_mask
        return current_pattern_bits == target_pattern_bits

    @staticmethod
    def get_pattern_mask(pattern_letters):
        """Calcula la máscara de bits para un conjunto de letras de posición."""
        mask = 0
        for letter in pattern_letters:
            if letter in code:
                mask |= (7 << code[letter][0])
        return mask

    # Métodos para la estructura del árbol de búsqueda
    def get_parent(self):
        return self.parent

    def get_depth(self):
        return self.depth

    # Método para desordenar el cubo
    def shuffle(self, n):
        """Desordena el cubo aplicando un número aleatorio de movimientos."""
        all_possible_actions = list(product([0, 1, 2], [0, 1], [0, 1]))
        for _ in range(n):
            random_action = choice(all_possible_actions)
            # Aplicar el movimiento mutando el objeto actual.
            # Se podría crear un estado temporal como en el código anterior para mayor seguridad,
            # pero la implementación de apply está diseñada para mutar self.
            self.apply(random_action)


    # Método para expandir un estado (generar sucesores)
    def expand(self):
        """Genera los estados sucesores aplicando todos los movimientos posibles."""
        all_possible_actions = list(product([0, 1, 2], [0, 1], [0, 1]))
        successors = []
        for action in all_possible_actions:
            # Crear un nuevo estado hijo aplicando la acción sobre el estado actual (self)
            child_state = RubikPuzzle(parent=self, action=action, depth=self.depth + 1)
            successors.append(child_state)

        # Filtrar el padre inmediato para evitar ciclos de 2 movimientos
        return list(filter(lambda x: (x != self.parent), successors))

In [3]:
# ----------- Funciones de Utilidad y Algoritmos de Búsqueda -----------

# Función para reconstruir la trayectoria desde el estado meta
def trajectory(end):
    """
    Reconstruye la ruta desde el estado meta hasta el estado inicial.
    """
    if end is None:
        return None
    sequence = deque()
    current = end
    while current:
        sequence.appendleft(current) # Añadir al inicio de la deque
        current = current.get_parent()
    return list(sequence)


# Algoritmo A* (Modificado para retornar historia de estructuras de memoria)
class AStar:
    """
    Implementación del algoritmo A* modificado para registrar el tamaño
    de la agenda (cola de prioridad) y el conjunto de expandidos en cada iteración.
    """
    @staticmethod
    def search(origen, stop, g, h):
        """
        Búsqueda informada A*.
        :return: Una tupla (ruta, historia) o (None, historia).
                 Historia es lista de (tamaño_agenda, tamaño_expandidos).
        """
        historia = [] # Lista para registrar (len(agenda), len(expandidos))

        agenda = [] # Cola de prioridad (implementada con lista y heapq)
        expandidos = set() # Conjunto de estados ya expandidos completamente

        # Caso base: el origen es la meta
        if stop(origen):
            # Registrar el estado inicial de memoria (estructuras vacías) y el estado final
            historia.append((0, 0))
            historia.append((0, 0)) # Estado final: agenda vacía, 0 expandidos (si la meta es el origen)
            return ([origen], historia)

        # Función de evaluación f(s) = g(s) + h(s)
        f = lambda s: g(s) + h(s)

        # Agregar el estado origen a la agenda
        heapq.heappush(agenda, (f(origen), origen))

        # Bucle principal de A*
        while agenda:
            # --- Registrar el estado actual de memoria ---
            # Registramos el tamaño ANTES de sacar el nodo para ver el estado de la agenda
            # y expandidos en este paso del proceso.
            historia.append((len(agenda), len(expandidos)))

            # Sacar el nodo con menor f de la agenda
            prioridad_actual, nodo_actual = heapq.heappop(agenda)

            # Si ya habíamos expandido completamente este nodo, lo saltamos (optimización con set de cerrados)
            # Esta simple verificación funciona con costo uniforme y heurística consistente.
            if nodo_actual in expandidos:
                 continue

            # --- Verificar si el nodo actual es la meta ---
            if stop(nodo_actual):
                # Si es la meta, reconstruimos la ruta y retornamos la historia completa.
                # Registrar el estado de memoria en el paso donde se encontró la meta (después del pop, antes del add a expandidos)
                # Los tamaños de agenda y expandidos ya fueron registrados al inicio del bucle *antes* de este pop.
                # Podríamos registrar un último punto reflejando el estado final de memoria después de encontrar la meta,
                # pero la interpretación de la gráfica es más clara si el punto final muestra el tamaño *en el paso exitoso*.
                return (trajectory(nodo_actual), historia)

            # --- Expandir el nodo actual ---
            # Añadir el nodo actual al conjunto de expandidos.
            expandidos.add(nodo_actual) # Ahora este nodo está "cerrado"

            # Generar sucesores
            for sucesor in nodo_actual.expand():
                # Solo consideramos agregar sucesores si no han sido completamente expandidos (no están en 'expandidos')
                # Con costo uniforme y heurística consistente, la primera vez que llegamos a un nodo es óptima a ese nodo,
                # por lo que no necesitamos actualizar su costo si ya está en la agenda ('open set').
                if sucesor not in expandidos:
                    f_sucesor = sucesor.get_depth() + h(sucesor)
                    heapq.heappush(agenda, (f_sucesor, sucesor))
                    # Nota: Si el sucesor ya estuviera en la agenda con un costo mayor,
                    # una implementación completa de A* en general necesitaría actualizar su prioridad.
                    # Pero con heapq y esta estructura simplificada, simplemente agregamos el duplicado.
                    # El manejo del set 'expandidos' asegura que solo procesemos el mejor camino al sacarlo.

        # Si el bucle termina y la agenda está vacía, significa que no hay ruta a la meta.
        # Registrar el estado final de memoria (agenda vacía, set de expandidos con todos los nodos explorados)
        historia.append((len(agenda), len(expandidos)))
        return (None, historia)


# Algoritmo IDA* (Implementación con búsqueda de profundidad limitada recursiva)
class IDAStar:
    """
    Implementación del algoritmo A* con profundidad iterada (Iterative Deepening A*).
    Utiliza una búsqueda de profundidad limitada (DLS) recursiva con una cota F creciente.
    Eficiente en memoria, completo y óptimo con heurística admisible y costo uniforme.
    """

    @staticmethod
    def search(start, stop, g, h):
        """
        Búsqueda A* con profundidad iterada.
        :param start: estado inicial.
        :param stop: función de paro (meta).
        :param g: función de costo acumulado (profundidad).
        :param h: función heurística (estimación a la meta).
        :return: Una tupla (ruta, historia) o (None, historia).
                 Historia es lista de (tamaño_stack_dls, tamaño_visitados_ruta_actual) por paso de DLS.
        """
        historia = [] # Lista para registrar (len(stack_dls), len(visited_in_path))

        # Función auxiliar recursiva para la búsqueda de profundidad limitada (DLS)
        # Retorna (estado_meta_encontrado, minimo_f_que_excedio_la_cota)
        def depth_limited_search(node, limit, visited_in_current_path, stack_size_proxy):
            """
            Realiza una búsqueda tipo DFS recursiva con una cota F.
            :param node: Nodo actual.
            :param limit: Cota máxima permitida para el valor f(nodo).
            :param visited_in_current_path: Set de nodos en el camino actual de la recursión (para detectar ciclos).
            :param stack_size_proxy: Un contador o lista que simula el tamaño del stack de llamadas.
                                    Pasado por referencia o globalmente para poder registrar su tamaño.
                                    Aquí usaremos len(visited_in_current_path) como proxy del tamaño del stack
                                    más un contador global para las llamadas totales.
            :return: Una tupla (estado_meta_encontrado, minimo_f_que_excedio_la_cota).
            """
            # --- Registrar el estado actual de memoria para este paso de DLS ---
            # En una DLS recursiva, la "agenda" es el stack de llamadas implícito.
            # Su tamaño es la profundidad de la recursión actual.
            # visited_in_current_path es el set para evitar ciclos en el camino actual.
            # historia.append((profundidad_recursión, len(visited_in_current_path)))
            # Para simplificar y seguir la idea del código original de medir len(deque) y len(set):
            # Usaremos el tamaño del set `visited_in_current_path` y un contador global para las "iteraciones".
            # O, más simple, registramos (len(visited_set), depth_actual) en la historia.
            # Adaptamos el registro de historia para IDA*: (tamaño_set_visitados_en_ruta_actual, profundidad_actual_del_nodo).
            # Esto es una proxy razonable para la memoria.

            historia.append((len(visited_in_current_path), node.get_depth()))


            # Calcular el valor f del nodo actual
            f_node = g(node) + h(node)

            # --- Poda por cota ---
            if f_node > limit:
                return None, f_node # Esta rama excede la cota, retornamos el valor f

            # --- Condición de meta ---
            if stop(node):
                return node, limit # ¡Meta encontrada dentro de la cota!

            # El mínimo valor f encontrado en los hijos que excedió la cota
            min_next_limit = math.inf

            # --- Expandir el nodo actual ---
            # Añadir el nodo al set de visitados en la ruta actual para detectar ciclos DLS.
            visited_in_current_path.add(node)

            # Generar sucesores y ordenarlos por f (optimización heurística dentro de DLS)
            successors = sorted(node.expand(), key=lambda s: g(s) + h(s))

            for child in successors:
                # Evitar ciclos en la ruta actual del DFS
                if child not in visited_in_current_path:
                    # Llamada recursiva para el hijo
                    found_target, next_limit_candidate = depth_limited_search(
                        child, limit, visited_in_current_path, stack_size_proxy # stack_size_proxy no se usa directamente aquí, solo para consistencia de firma si fuera necesario
                    )

                    # Propagar la meta encontrada hacia arriba
                    if found_target:
                        return found_target, limit

                    # Actualizar el mínimo valor f que excedió la cota
                    if next_limit_candidate is not None:
                        min_next_limit = min(min_next_limit, next_limit_candidate)

            # --- Al salir del bucle for ---
            # Eliminar el nodo actual del set de visitados en la ruta actual al retroceder en la recursión.
            visited_in_current_path.remove(node)

            # Retornar el mínimo valor f que excedió la cota en este subárbol, o la cota actual
            # si ningún hijo superó la cota (significa que la rama se agotó dentro de la cota).
            # Si min_next_limit sigue siendo math.inf, significa que todos los hijos estaban
            # ya en visited_in_current_path o f_child > limit.
            # El valor a retornar para la próxima cota es el mínimo f que superó la cota en cualquier hijo.
            return None, min_next_limit # Retorna None (no encontró meta) y el próximo límite candidato.


        # --- Lógica principal de IDA* ---

        # La cota inicial es la heurística del nodo de inicio
        cota_actual = h(start)

        # Bucle principal de Iterative Deepening
        while True:
            # Inicializar variables para la nueva iteración de profundidad limitada
            # El set de visitados en la ruta actual se reinicia en cada iteración de IDA*.
            visited_in_current_path_dls = set()
            # Un proxy para el tamaño del stack de DLS (no estrictamente necesario si registramos profundidad y set)
            stack_size_proxy = 0 # No usado directamente en esta versión recursiva para el registro de historia.

            # Ejecutar la búsqueda de profundidad limitada recursiva desde el inicio
            found_target, minimo_encontrado_en_iteracion = depth_limited_search(
                start, cota_actual, visited_in_current_path_dls, stack_size_proxy
            )

            # Si la búsqueda de profundidad limitada encontró la meta
            if found_target:
                # Reconstruir la ruta y retornar la historia completa.
                return (trajectory(found_target), historia)

            # Si no se encontró la meta, la próxima cota es el mínimo valor f que excedió la cota actual.
            cota_actual = minimo_encontrado_en_iteracion

            # Si el mínimo encontrado es infinito, significa que no hay más nodos alcanzables.
            if cota_actual == math.inf:
                # No hay ruta a la meta
                return (None, historia)

            # La próxima iteración de IDA* comenzará con la nueva cota.
            # El set visited_in_current_path_dls se reinicia para la nueva iteración.

In [ ]:
# ----------- Implementación de Heurística para Rubik (Pattern Databases) -----------

class PatternBasedHeuristic:
    """
    Implementación de Heurística para el cubo de Rubik basada en bases de datos de patrones (PDB).
    Construye la base de datos mediante BFS sobre estados completos, registrando la profundidad
    de las configuraciones de patrón encontradas.
    """
    def __init__(self, objective=None, depth=6, pattern_letters=None):
        """
        Crea la base de datos de patrones.
        :param objective: El estado meta (RubikPuzzle resuelto por defecto).
        :param depth: La profundidad máxima de los estados a incluir en la PDB.
        :param pattern_letters: Cadena de letras que definen las posiciones del patrón.
        """
        pattern_name = pattern_letters or "Defecto (Esquinas)"
        print(f'Calculando base de datos de patrón "{pattern_name}" hasta profundidad {depth}...')

        if objective is None:
            objective = RubikPuzzle()

        # Cola para el BFS de estados completos del cubo
        agenda_bfs = deque([objective])
        # Set para rastrear los estados completos ya explorados en el BFS
        explored_bfs = {objective}
        # Diccionario para almacenar la PDB: {config_bits_patron: profundidad_minima_del_estado_completo}
        self.patterns = {}

        self.depth_limit = depth
        self.pattern_letters = pattern_letters
        # Calcula la máscara de bits para aislar el patrón
        self.pattern_mask = RubikPuzzle.get_pattern_mask(self.pattern_letters)

        # Añadir la configuración del patrón del estado objetivo (profundidad 0)
        objective_pattern_conf = objective.configuration & self.pattern_mask
        self.patterns[objective_pattern_conf] = 0

        nodes_processed = 0
        # Bucle principal del BFS sobre estados completos
        while agenda_bfs:
            current_node = agenda_bfs.popleft() # Sacar el estado más antiguo (BFS)
            nodes_processed += 1

            # Mostrar progreso
            if nodes_processed % 50000 == 0:
                print(f"  PDB '{pattern_name}': Procesados {nodes_processed} nodos. Agenda BFS: {len(agenda_bfs)}, Patrones encontrados: {len(self.patterns)}")

            current_depth = current_node.get_depth()

            # Si la profundidad del estado actual excede el límite de la PDB, no expandimos más desde aquí.
            if current_depth >= self.depth_limit:
                continue

            # Expandir el estado completo actual
            for child in current_node.expand():
                # Si este estado hijo completo no ha sido explorado en el BFS
                if child not in explored_bfs:
                    explored_bfs.add(child)
                    agenda_bfs.append(child) # Añadir a la cola para explorarlo después

                    # Obtener la configuración del patrón para este estado hijo
                    child_pattern_conf = child.configuration & self.pattern_mask

                    # Si esta configuración de patrón es nueva, la añadimos a la PDB
                    # La profundidad asociada es la profundidad del estado COMPLETO que la generó.
                    # Esto garantiza que la heurística sea admisible.
                    if child_pattern_conf not in self.patterns:
                        self.patterns[child_pattern_conf] = child.get_depth()

        print(f"Base de datos de patrón '{pattern_name}' construida. Tamaño final: {len(self.patterns)} patrones.")


    def heuristic(self, puzzle):
        """
        Calcula la heurística (distancia estimada al objetivo) usando la base de datos de patrones.
        :param puzzle: El estado actual del cubo (objeto RubikPuzzle).
        :return: La profundidad mínima pre-calculada para el patrón del cubo actual,
                 o depth_limit + 1 si el patrón no se encontró en la base de datos (para garantizar admisibilidad).
        """
        # Obtener la configuración de bits del patrón para el cubo actual
        puzzle_pattern_conf = puzzle.configuration & self.pattern_mask

        # Buscar esta configuración de patrón en la base de datos.
        # Retornar la profundidad almacenada si se encuentra.
        # Si el patrón no está en la PDB (porque está más allá de depth_limit o inalcanzable),
        # retornamos un valor que garantiza admisibilidad (depth_limit + 1 es una opción común).
        return self.patterns.get(puzzle_pattern_conf, self.depth_limit + 1)


# ----------- Funciones para Graficar Resultados -----------

# Función para dibujar gráfica de memoria para un solo algoritmo (opcional, la comparativa es más útil)
def dibujar_grafica(historia, algoritmo):
    """
    Dibuja una gráfica del tamaño de estructuras de memoria para un solo algoritmo.
    Útil para inspeccionar el comportamiento individual.
    """
    if not historia:
        print(f"No hay datos de historia para graficar para {algoritmo}")
        return

    # Adaptar etiquetas según el algoritmo
    if algoritmo.startswith('A*') or algoritmo.startswith('UCS'):
         label1, label2 = 'Agenda (Heap)', 'Expandidos (Set)'
         # En A*, cada punto de historia es un nodo sacado del heap
         xlabel = 'Nodos Sacados del Heap (Iteración de A*)'
    elif algoritmo.startswith('IDA*'):
         label1, label2 = 'Visitados en Ruta Actual (Set)', 'Profundidad Actual del Nodo' # Ajustamos etiquetas para IDA*
         # En IDA*, cada punto de historia es un paso en la DLS recursiva
         xlabel = 'Pasos en Búsqueda de Profundidad Limitada (Iteración de DLS)'
    else:
         label1, label2 = 'Estructura 1', 'Estructura 2'
         xlabel = 'Iteración'


    tamano_estructura1, tamano_estructura2 = zip(*historia)
    # El "total" para IDA* en esta representación no suma las estructuras directamente
    # sino que muestra las dos métricas (tamaño set, profundidad) por separado en el mismo plot si se desea.
    # Para mantener la consistencia con la gráfica comparativa, graficaremos solo Elem1 + Elem2.
    # Para IDA*, Elem1 es len(set), Elem2 es depth.
    # El "total" aquí puede no tener una interpretación directa como "memoria total" para IDA*.
    tamano_total =  [s1 + s2 for s1, s2 in historia]


    plt.figure(figsize=(10, 6))
    line1, = plt.plot(tamano_estructura1, label=label1, color='blue')
    # line2, = plt.plot(tamano_estructura2, label=label2, color='red') # Opcional: graficar la 2da estructura
    # line3, = plt.plot(tamano_total, label='Total (Estruc 1 + Estruc 2)', color='green', linestyle='--') # Opcional: graficar el total

    plt.legend(handles=[line1]) # Ajustar handles si se grafican más líneas
    plt.xlabel(xlabel)
    plt.ylabel('Tamaño / Profundidad')
    plt.title(f'Consumo de "Memoria" por {algoritmo}')
    plt.grid(True)
    plt.show()


# Función para dibujar gráfica comparativa del tamaño total de estructuras de memoria
def dibujar_graficas_comparativas_memoria(historias_algoritmos, nombres_algoritmos):
    """
    Dibuja gráficas comparativas del tamaño total de las estructuras de "memoria"
    para múltiples algoritmos en el mismo gráfico.
    """
    plt.figure(figsize=(12, 8))

    colors = ['blue', 'red', 'green', 'purple', 'orange']
    linestyles = ['-', '--', '-.', ':', '-']

    plotted_any = False
    for i, (historia, nombre_algoritmo) in enumerate(zip(historias_algoritmos, nombres_algoritmos)):
        if not historia:
            print(f"No hay datos de historia para {nombre_algoritmo}, saltando gráfica.")
            continue

        # Calcular el "tamaño total" para cada algoritmo según su estructura de historia
        # Para A*/UCS: len(agenda) + len(expandidos)
        # Para IDA*: len(visited_in_current_path) + depth_actual (aunque la suma no es intuitiva, es para comparar la escala de los números registrados)
        # O simplemente graficamos el tamaño de la estructura más grande para cada uno: len(expandidos) para A*, len(visited_in_current_path) para IDA*.
        # La forma original (len(estruc1)+len(estruc2)) se usó para el rompecabezas del 15. Mantenemos esa forma para consistencia.
        # tamano_total = [s1 + s2 for s1, s2 in historia]
        # O graficar solo el tamaño de la estructura principal que crece (expandidos para A*, set para IDA*):
        tamano_principal = [s2 for s1, s2 in historia] # El segundo elemento en la tupla de historia

        plt.plot(tamano_principal, label=f'{nombre_algoritmo} (Tamaño Estructura Principal)', color=colors[i % len(colors)], linestyle=linestyles[i % len(linestyles)])
        plotted_any = True

    if plotted_any:
        plt.xlabel('Paso/Iteración (Relativo)')
        plt.ylabel('Tamaño de la Estructura Principal')
        plt.title('Comparación de "Memoria" (Tamaño de Estructuras Clave) entre Algoritmos')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print("No hay suficientes datos de historia disponibles para generar la gráfica comparativa de memoria.")


# ----------- Código principal para realizar la tarea -----------

# Este bloque contiene la lógica principal para configurar y ejecutar el estudio.

# Definir el estado objetivo (cubo resuelto)
estado_meta = RubikPuzzle(configuration=initial_conf)
stop_function = lambda s: s == estado_meta
cost_function = lambda s: s.get_depth() # Costo uniforme (profundidad)

# --- Parte 1: Preparar heurísticas (Bases de Datos de Patrones) ---
# Construir las bases de datos de patrones a una profundidad manejable (ej. 6).
# Esto puede tomar tiempo.

# Patrón de esquinas
pattern_corners_letters = "ACGIJLgiMÑjlOQmñRToqrtxz"
# Patrón de bordes + centro
pattern_edges_centers_letters = "BDEFHKUVWhNXYZkPabcnSdefpsuvwy"

# Profundidad para las bases de datos de patrones
pdb_depth = 6

print("\n--- Construyendo Bases de Datos de Patrones ---")
pdb_corners = PatternBasedHeuristic(objective=estado_meta, depth=pdb_depth, pattern_letters=pattern_corners_letters)
pdb_edges_centers = PatternBasedHeuristic(objective=estado_meta, depth=pdb_depth, pattern_letters=pattern_edges_centers_letters)

# Definir la heurística combinada
h_corners = pdb_corners.heuristic
h_edges_centers = pdb_edges_centers.heuristic
h_combined = lambda s: max(h_corners(s), h_edges_centers(s))
print("Heurística combinada basada en PDBs configurada.")

# --- Parte 2: Seleccionar un cubo de prueba ---
# Generar un cubo desordenado que sea resoluble a profundidad <= 10.
# Desordenar con N movimientos aleatorios es una forma común, aunque la distancia óptima puede ser < N.
num_shuffle_moves = 9 # Ajusta este número para probar diferentes dificultades

print(f"\n--- Generando Cubo Inicial Desordenado (con {num_shuffle_moves} movimientos) ---")
cubo_inicio = RubikPuzzle(configuration=initial_conf)
cubo_inicio.shuffle(num_shuffle_moves)

print("\nCubo inicial para el estudio comparativo:")
print(cubo_inicio)
print("-" * 30) # Separador visual

# --- Parte 3: Realizar el estudio comparativo ---
print("\n--- Iniciando Estudio Comparativo (Objetivo: Profundidad <= 10) ---")

# Listas para almacenar resultados para la tabla resumen y gráficas
tiempos = {}
longitudes_ruta = {}
iteraciones_historia = {} # Número de puntos en la historia
completez = {}
historias_para_grafica_memoria = []
nombres_para_grafica_memoria = []

# Algoritmo 1: UCS (Uniform Cost Search)
print("\nEjecutando UCS (A* con h=0)...")
h_ucs = lambda s: 0
start_time_ucs = time.time()
ruta_ucs, historia_ucs = AStar.search(cubo_inicio, stop_function, cost_function, h_ucs)
end_time_ucs = time.time()
tiempo_ucs = end_time_ucs - start_time_ucs

tiempos['UCS'] = tiempo_ucs
iteraciones_historia['UCS'] = len(historia_ucs)
if ruta_ucs:
    longitudes_ruta['UCS'] = len(ruta_ucs) - 1
    completez['UCS'] = 'Sí'
    historias_para_grafica_memoria.append(historia_ucs)
    nombres_para_grafica_memoria.append('UCS (A* with h=0)')
    print(f"UCS encontró una solución en {longitudes_ruta['UCS']} movimientos.")
else:
    longitudes_ruta['UCS'] = 'No encontrada'
    completez['UCS'] = 'No'
    print(f"UCS no encontró una solución.")

# Algoritmo 2: A* (con heurística combinada)
print("\nEjecutando A* con heurística combinada...")
start_time_astar = time.time()
ruta_astar, historia_astar = AStar.search(cubo_inicio, stop_function, cost_function, h_combined)
end_time_astar = time.time()
tiempo_astar = end_time_astar - start_time_astar

tiempos['A*'] = tiempo_astar
iteraciones_historia['A*'] = len(historia_astar)
if ruta_astar:
    longitudes_ruta['A*'] = len(ruta_astar) - 1
    completez['A*'] = 'Sí'
    historias_para_grafica_memoria.append(historia_astar)
    nombres_para_grafica_memoria.append('A* (Combined Heuristic)')
    print(f"A* encontró una solución en {longitudes_ruta['A*']} movimientos.")
else:
    longitudes_ruta['A*'] = 'No encontrada'
    completez['A*'] = 'No'
    print(f"A* no encontró una solución.")


# Algoritmo 3: IDA* (con heurística combinada)
print("\nEjecutando IDA* con heurística combinada...")
start_time_idastar = time.time()
ruta_idastar, historia_idastar = IDAStar.search(cubo_inicio, stop_function, cost_function, h_combined)
end_time_idastar = time.time()
tiempo_idastar = end_time_idastar - start_time_idastar

tiempos['IDA*'] = tiempo_idastar
iteraciones_historia['IDA*'] = len(historia_idastar)
if ruta_idastar:
    longitudes_ruta['IDA*'] = len(ruta_idastar) - 1
    completez['IDA*'] = 'Sí'
    # La historia de IDA* es mucho más larga en # puntos, pero los valores son más bajos.
    # Puede que necesitemos un manejo diferente para graficarla junto a A*/UCS.
    # La función dibujar_graficas_comparativas_memoria intenta acomodarlas por # de puntos.
    historias_para_grafica_memoria.append(historia_idastar)
    nombres_para_grafica_memoria.append('IDA* (Combined Heuristic)')
    print(f"IDA* encontró una solución en {longitudes_ruta['IDA*']} movimientos.")
else:
    longitudes_ruta['IDA*'] = 'No encontrada'
    completez['IDA*'] = 'No'
    print(f"IDA* no encontró una solución.")


# --- Parte 4: Presentar Resultados y Generar Gráficas Comparativas ---
print("\n--- Resultados del Estudio Comparativo ---")
print(f"{'Algoritmo':<25} | {'Tiempo (s)':<15} | {'Longitud Ruta':<15} | {'Pasos Historia':<20} | {'Completez':<10}")
print("-" * 100)

# Presentar resultados de cada algoritmo
for algo_name in ['UCS', 'A*', 'IDA*']:
    tiempo_val = tiempos.get(algo_name, 'N/A')
    ruta_len_val = longitudes_ruta.get(algo_name, 'No Ejecutado')
    hist_len_val = iteraciones_historia.get(algo_name, 'N/A')
    completez_val = completez.get(algo_name, 'N/A')

    # Formatear tiempo si es un número
    tiempo_str = f"{tiempo_val:.4f}" if isinstance(tiempo_val, (int, float)) else tiempo_val

    print(f"{algo_name:<25} | {tiempo_str:<15} | {str(ruta_len_val):<15} | {str(hist_len_val):<20} | {completez_val:<10}")


# Generar la gráfica comparativa de memoria para los algoritmos que encontraron solución
print("\n--- Generando Gráfica Comparativa de Memoria ---")
if historias_para_grafica_memoria:
    dibujar_graficas_comparativas_memoria(historias_para_grafica_memoria, nombres_para_grafica_memoria)
else:
    print("No se puede generar la gráfica comparativa de memoria ya que ningún algoritmo encontró una solución para graficar.")


# --- Parte 5: Proponer y aplicar algoritmo modificado/combinado ---
print("\n--- Parte 5: Tu Propuesta de Algoritmo Modificado/Combinado ---")
print("Aquí debes implementar y describir tu modificación o combinación de algoritmos/heurísticas.")
print("Reemplaza este texto con la explicación y el código de tu propuesta.")
print("\nPor ejemplo, podrías intentar:")
print("- Implementar una heurística diferente.")
print("- Combinar heurísticas de otra manera.")
print("- Modificar un algoritmo de búsqueda (ej. IDA* con re-expansión controlada).")
print("- Aplicar una búsqueda local después de alcanzar cierta profundidad.")
print("\nPara probar tu modificación, crea un nuevo cubo de prueba si es necesario y ejecuta tu algoritmo sobre él.")

# Ejemplo de placeholder para tu implementación (elimina o modifica según tu propuesta):
# print("\nEjecutando mi Algoritmo Modificado (Placeholder)...")
# # Aquí iría el código para tu algoritmo modificado
# # start_time_mod = time.time()
# # ... tu_algoritmo_modificado.search(cubo_inicio, ...)
# # end_time_mod = time.time()
# # tiempo_mod = end_time_mod - start_time_mod
# # print(f"Tiempo de ejecución de mi algoritmo modificado: {tiempo_mod:.4f} segundos.")
# # Si tu algoritmo modificado también registra historia, puedes graficarla.
# # dibujar_grafica(historia_mod, 'Mi Algoritmo Modificado')


--- Construyendo Bases de Datos de Patrones ---
Calculando base de datos de patrón "ACGIJLgiMÑjlOQmñRToqrtxz" hasta profundidad 6...
  PDB 'ACGIJLgiMÑjlOQmñRToqrtxz': Procesados 50000 nodos. Agenda BFS: 418575, Patrones encontrados: 147946
  PDB 'ACGIJLgiMÑjlOQmñRToqrtxz': Procesados 100000 nodos. Agenda BFS: 836708, Patrones encontrados: 242837
  PDB 'ACGIJLgiMÑjlOQmñRToqrtxz': Procesados 150000 nodos. Agenda BFS: 833926, Patrones encontrados: 247044
  PDB 'ACGIJLgiMÑjlOQmñRToqrtxz': Procesados 200000 nodos. Agenda BFS: 783926, Patrones encontrados: 247044
  PDB 'ACGIJLgiMÑjlOQmñRToqrtxz': Procesados 250000 nodos. Agenda BFS: 733926, Patrones encontrados: 247044
  PDB 'ACGIJLgiMÑjlOQmñRToqrtxz': Procesados 300000 nodos. Agenda BFS: 683926, Patrones encontrados: 247044
  PDB 'ACGIJLgiMÑjlOQmñRToqrtxz': Procesados 350000 nodos. Agenda BFS: 633926, Patrones encontrados: 247044
  PDB 'ACGIJLgiMÑjlOQmñRToqrtxz': Procesados 400000 nodos. Agenda BFS: 583926, Patrones encontrados: 247044
  P

## Análisis de Resultados y Conclusiones

**(Completa esta sección basándote en la salida del código y las gráficas generadas)**

Describe y compara el rendimiento de los tres algoritmos (UCS, A*, IDA*) según los criterios de la tarea:

-   **Memoria:** Analiza las gráficas generadas. ¿Qué algoritmo consume más memoria? ¿Cuál menos? Explica por qué, relacionándolo con cómo funciona cada algoritmo (uso de agenda, set de expandidos, stack de recursión, reinicio en cada iteración de IDA*).
-   **Tiempo:** Compara los tiempos de ejecución de la tabla de resultados. ¿Cuál es el más rápido? ¿Cuál el más lento? Explica cómo la heurística informada afecta el tiempo de búsqueda en comparación con la búsqueda no informada (UCS).
-   **Calidad de la Solución:** Observa la longitud de la ruta encontrada para cada algoritmo en la tabla. Dado que se usan algoritmos óptimos con heurística admisible, la longitud debería ser la mínima (si se encontró solución). Comenta si todos encontraron la ruta óptima y por qué.
-   **Completez:** Indica si cada algoritmo fue capaz de encontrar una solución para el cubo de prueba. Explica bajo qué condiciones son completos en teoría.

Resume cuál consideras el mejor algoritmo para resolver el cubo de Rubik a una profundidad <= 10, justificando tu elección con los resultados obtenidos.

---

## Propuesta de Modificación y Aplicación

**(Completa esta sección con tu trabajo de la Parte 5 del código)**

Describe detalladamente la modificación al algoritmo o heurística que has propuesto. Explica la lógica detrás de tu propuesta y cómo crees que podría mejorar el rendimiento o la calidad de la solución (aunque para calidad con algoritmos óptimos es más difícil mejorarla directamente, podría ser sobre la eficiencia).

Presenta el código que has implementado en la Parte 5.

Describe el cubo de prueba que utilizaste para evaluar tu algoritmo modificado y los resultados obtenidos (Tiempo, Memoria - si aplicaste mediciones, Longitud de Ruta).

Analiza si tu modificación tuvo el efecto esperado y compara su rendimiento con los algoritmos base estudiados previamente.